Some models have been created before the addition of metrics like MAE and MSE. This notebook adds them to the metrics.json.

How to use this notebook?

Locate this notebook in the directory where the model, metrics.json and data folder are located (Example: locate this notebook in results\1-For-Microtrips\Physics-and-MachineLearning). Then, run this notebook.

In [15]:
import os
import pandas as pd
import torch
import json
import joblib

# Get test data

In [2]:
def get_filepaths_with_extension(file_extension=".csv", directory="."):
    output = []
    
    for file in os.listdir(directory):  
        # Check if the file has the required extension
        if file.endswith(file_extension):
            output.append(f"{directory}/{file}") 
            
    return output

In [3]:
all_data_filepaths = get_filepaths_with_extension(".csv", "test_data_and_model_output")
all_data_filepaths[:3]

['test_data_and_model_output/file-0.csv',
 'test_data_and_model_output/file-1.csv',
 'test_data_and_model_output/file-2.csv']

In [4]:
# read all model results
# each model will be represented by an index, and inside that index, all the dfs that contain the test results
lst_dfs = []    

for idx_df in range(len(all_data_filepaths)):
    lst_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in all_data_filepaths]

In [5]:
len(lst_dfs)

10

# Calculate additional losses

In [16]:
def get_file(filename_start, file_extension=".csv", directory="."):

    for file in os.listdir(directory):  
        # Check if the file has the required extension
        if file.endswith(file_extension) and file.startswith(filename_start):
            return f"{directory}/{file}"

In [18]:
# load used scaler for y and y_hat
y_scaler_filename = get_file("y-scaler", file_extension=".pkl")
y_scaler = joblib.load(y_scaler_filename) 

In [6]:
# Loss function
def rmse_loss(y_true, y_pred):
    mse_loss = torch.nn.functional.mse_loss(y_pred, y_true) 
    return torch.sqrt(mse_loss)  
    
# loss_function =  torch.nn.MSELoss()  
loss_function = rmse_loss 

In [22]:
mse_loss_function = torch.nn.functional.mse_loss
mae_loss_function = torch.nn.functional.l1_loss

test_mse_loss = 0
test_mae_loss = 0
test_loss = 0

for df in lst_dfs:
    # get necessary data for calculation of metrics
    ground_truth = df["SoC [%]"].values
    model_output_column_name = df.columns[-1]
    predictions = df[model_output_column_name].values

    # reshape predictions and ground truth 
    predictions = predictions.reshape(-1, 1)
    ground_truth = ground_truth.reshape(-1, 1)

    # scale predictions and ground truth as in original evaluation
    predictions = y_scaler.transform(predictions)
    ground_truth = y_scaler.transform(ground_truth)

    # Convert to PyTorch tensors
    predictions = torch.tensor(predictions, dtype=torch.float32)
    ground_truth = torch.tensor(ground_truth, dtype=torch.float32)
    
    # calculate additional metrics
    test_loss += loss_function(predictions, ground_truth).item()
    test_mse_loss += mse_loss_function(predictions, ground_truth).item()
    test_mae_loss += mae_loss_function(predictions, ground_truth).item()

# print main loss
avg_test_loss = test_loss / len(lst_dfs)
print(f"2.Test Loss (RMSE) - Metric used for training: {avg_test_loss:.6f}")

# print additional metrics
avg_mse_loss = test_mse_loss / len(lst_dfs)
avg_mae_loss = test_mae_loss / len(lst_dfs)
print(f"2.Test Loss (MSE): {avg_mse_loss:.6f}")
print(f"2.Test Loss (MAE): {avg_mae_loss:.6f}")

2.Test Loss (RMSE) - Metric used for training: 0.129499
2.Test Loss (MSE): 0.019651
2.Test Loss (MAE): 0.113145


# Export metrics to existing file or create new file if no metrics.json found

In [8]:
def load_json_file(file_path):
    """
    Load a JSON file and return its contents as a dictionary.
    :param file_path: Path to the JSON file.
    :return: Dictionary containing JSON data.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"The file '{file_path}' was not found. Returning empty dictionary.")
        return {}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [9]:
file_path = "metrics.json"
metrics = load_json_file(file_path)

In [10]:
# add calculated metrics to imported metrics dictionary
metrics["2.test set loss (RMSE) - used for training"] = avg_test_loss
metrics["2.test set loss (MSE)"] = avg_mse_loss
metrics["2.test set loss (MAE)"] = avg_mae_loss

In [11]:
# export metrics
with open(file_path, 'w') as file:
    json.dump(metrics, file)